# 0. 데이터 준비
- 캐글 __Dogs vs. Cats__ 데이터셋 
- https://www.kaggle.com/c/dogs-vs-cats/data 에서 다운로드 가능 (캐글 로그인 필요)
- 다운로드 받은 데이터셋 파일(dogs-vs-cats.zip) 압축해제 및 그 안의 train.zip 파일 압축해제

In [1]:
import glob
import numpy as np
import pandas as pd
import os
import shutil
from utils import log_progress

np.random.seed(42)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
## 데이터 경로
files = glob.glob('train/*')
files[:10]

In [ ]:
## 파일명을 통해 cat 파일, dog 파일 분리
cat_files = [fn for fn in files if 'cat' in fn]
dog_files = [fn for fn in files if 'dog' in fn]

len(cat_files), len(dog_files)

In [ ]:
# 이렇게 한번에 해도 됨...
# cat_files = glob.glob('train/cat.*') 
# dog_files = glob.glob('train/dog.*') 

In [ ]:
## 학습데이터/검증데이터/테스트데이터 분리 
# cf) sklearn의 sklearn.model_selection.train_test_split() 사용하면 편리 (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

cat_train = np.random.choice(cat_files, size=1500, replace=False)
dog_train = np.random.choice(dog_files, size=1500, replace=False)
cat_files = list(set(cat_files) - set(cat_train))
dog_files = list(set(dog_files) - set(dog_train))

cat_val = np.random.choice(cat_files, size=500, replace=False)
dog_val = np.random.choice(dog_files, size=500, replace=False)
cat_files = list(set(cat_files) - set(cat_val))
dog_files = list(set(dog_files) - set(dog_val))

cat_test = np.random.choice(cat_files, size=500, replace=False)
dog_test = np.random.choice(dog_files, size=500, replace=False)

print('Cat datasets:', cat_train.shape, cat_val.shape, cat_test.shape)
print('Dog datasets:', dog_train.shape, dog_val.shape, dog_test.shape)

In [ ]:
## 학습데이터/검증데이터/테스트데이터 각각 별도의 파일을 만들어서 저장
train_dir = 'training_data'
val_dir = 'validation_data'
test_dir = 'test_data'

# cat, dog 데이터를 결합
train_files = np.concatenate([cat_train, dog_train]) 
validate_files = np.concatenate([cat_val, dog_val])
test_files = np.concatenate([cat_test, dog_test])

# 각 경로가 없으면 새로 만듦.
os.mkdir(train_dir) if not os.path.isdir(train_dir) else None
os.mkdir(val_dir) if not os.path.isdir(val_dir) else None
os.mkdir(test_dir) if not os.path.isdir(test_dir) else None

# 학습데이터/검증데이터/테스트데이터 경로에 원본 파일 복사
for fn in log_progress(train_files, name='Training Images'):
    shutil.copy(fn, train_dir)

for fn in log_progress(validate_files, name='Validation Images'):
    shutil.copy(fn, val_dir)
    
for fn in log_progress(test_files, name='Test Images'):
    shutil.copy(fn, test_dir)

# 1. 합성곱 신경망(CNN) 모델 처음부터 구축하기

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array, array_to_img

## 1) 데이터 불러오기

In [ ]:
## 학습 데이터 / 검증 데이터 준비
# 이미지의 가로,세로 사이즈는 150,150이고 빨강, 녹색, 파랑(RGB)인 3개의 채널이 있으므로 
# 출력되는 shape은 (데이터 갯수, 150, 150, 3)의 형태. 

IMG_DIM = (150, 150)

# 학습 데이터 
train_files = glob.glob('training_data/*')
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('/')[1].split('.')[0].strip() for fn in train_files]

# 검증 데이터
validation_files = glob.glob('validation_data/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[1].split('.')[0].strip() for fn in validation_files]

print('Train dataset shape:', train_imgs.shape, 
      '\nValidation dataset shape:', validation_imgs.shape) 

In [ ]:
## 데이터 샘플 확인
print(train_imgs[0].shape)
array_to_img(train_imgs[0])

In [ ]:
## 텍스트 클래스 레이블 인코딩
# cat : 0, dog : 1
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
validation_labels_enc = le.transform(validation_labels)

print('Before Encoding : ', train_labels[1495:1505], 
      '\nAfter Encoding : ', train_labels_enc[1495:1505])

## 2) 데이터 증강 

In [ ]:
train_files = glob.glob('training_data/*')
train_files[:10]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
## 데이터 증강된 고양이 이미지 샘플
img_id = 2595
cat_generator = train_datagen.flow(train_imgs[img_id:img_id+1], train_labels[img_id:img_id+1],
                                   batch_size=1)
cat = [next(cat_generator) for i in range(0,5)]
fig, ax = plt.subplots(1,5, figsize=(16, 6))
print('Labels:', [item[1][0] for item in cat])
l = [ax[i].imshow(cat[i][0][0]) for i in range(0,5)]

In [ ]:
## 데이터 증강된 강아지 이미지 샘플
img_id = 1991
dog_generator = train_datagen.flow(train_imgs[img_id:img_id+1], train_labels[img_id:img_id+1],
                                   batch_size=1)
dog = [next(dog_generator) for i in range(0,5)]
fig, ax = plt.subplots(1,5, figsize=(15, 6))
print('Labels:', [item[1][0] for item in dog])
l = [ax[i].imshow(dog[i][0][0]) for i in range(0,5)]

## 3) 모델 정의

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras import optimizers

In [ ]:
## 하이퍼 파라미터 설정
batch_size = 100 # 배치 사이즈 : 반복해서 모델에 전달되는 데이터의 수
num_classes = 2 # 클래스 개수 (cat, dog)
epochs = 100 # 에포크 : 학습하는동안 모델에 전달하는 전체 데이터 세트를 횟수
input_shape = (150, 150, 3) # 입력되는 이미지 사이즈 (가로사이즈, 세로사이즈, 채널 수)

In [ ]:
## CNN 모델 컴파일 및 요약 정보 확인
model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

model.summary()

## 4) 모델 학습

In [ ]:
## 모델 학습
train_generator = train_datagen.flow(train_imgs, train_labels_enc, batch_size=batch_size)
val_generator = val_datagen.flow(validation_imgs, validation_labels_enc, batch_size=batch_size)

history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=epochs,
                              validation_data=val_generator, validation_steps=50, verbose=1)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('CNN with Image Augmentation Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1, epochs+1))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 101, 10))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 101, 10))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
model.save('cats_dogs_cnn_img_aug.h5')

# 2. 사전 훈련된 CNN 모델에 전이학습 활용하기
- VGG-16 : https://arxiv.org/pdf/1409.1556.pdf

## 1) 데이터 불러오기 (위와 동일)

In [ ]:
## 학습 데이터 / 검증 데이터 준비
# 이미지의 가로,세로 사이즈는 150,150이고 빨강, 녹색, 파랑(RGB)인 3개의 채널이 있으므로 
# 출력되는 shape은 (데이터 갯수, 150, 150, 3)의 형태. 

IMG_DIM = (150, 150)

# 학습 데이터 
train_files = glob.glob('training_data/*')
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('/')[1].split('.')[0].strip() for fn in train_files]

# 검증 데이터
validation_files = glob.glob('validation_data/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[1].split('.')[0].strip() for fn in validation_files]

print('Train dataset shape:', train_imgs.shape, 
      '\nValidation dataset shape:', validation_imgs.shape) 

## 2) 데이터 증강 (위와 동일)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

## 3) 모델 정의

In [ ]:
## 하이퍼 파라미터 설정
batch_size = 30 # 배치 사이즈 : 반복해서 모델에 전달되는 데이터의 수
num_classes = 2 # 클래스 개수 (cat, dog)
epochs = 100 #100 # 에포크 : 학습하는동안 모델에 전달하는 전체 데이터 세트를 횟수
input_shape = (150, 150, 3) # 입력되는 이미지 사이즈 (가로사이즈, 세로사이즈, 채널 수)

In [ ]:
## 사전 학습된 VGG-16 모델 불러오기
import keras
from keras.models import Model
from keras.applications import vgg16

vgg = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=input_shape)

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)

vgg_model = Model(vgg.input, output)
vgg_model.trainable = False

for layer in vgg_model.layers:
    layer.trainable = False

vgg_model.summary()

In [ ]:
# VGG-16 모델의 층이 고정되었는지 확인
pd.set_option('max_colwidth', -1)

layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
# VGG-16의 합성곱 블록 4와 5가 학습 가능하도록 설정
vgg_model.trainable = True
set_trainable = False

for layer in vgg_model.layers:
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# VGG-16 모델의 층이 학습 가능한지 확인

layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.RMSprop(lr=1e-5), 
              metrics=['accuracy'])

model.summary()

## 4) 모델 학습

In [ ]:
train_generator = train_datagen.flow(train_imgs, train_labels_enc, batch_size=batch_size)
val_generator = val_datagen.flow(validation_imgs, validation_labels_enc, batch_size=batch_size)

history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=epochs,
                              validation_data=val_generator, validation_steps=50, verbose=1)


In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Pre-trained CNN (Transfer Learning) with Fine-Tuning & Image Augmentation Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1, epochs+1))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 101, 10))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 101, 10))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
model.save('cats_dogs_tlearn_finetune_img_aug_cnn.h5')

# 3. 모델 성능 평가

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.models import load_model
import model_evaluation_utils as meu

In [ ]:
img_aug_cnn = load_model('cats_dogs_cnn_img_aug.h5')
tl_img_aug_finetune_cnn = load_model('cats_dogs_tlearn_finetune_img_aug_cnn.h5')

In [ ]:
## 이미지 기본 모양
IMG_DIM = (150, 150)
input_shape = (150, 150, 3)
num2class_label_transformer = lambda l: ['cat' if x == 0 else 'dog' for x in l]
class2num_label_transformer = lambda l: [0 if x == 'cat' else 1 for x in l]

## 1) 샘플 테스트 이미지로 모델 예측

In [ ]:
sample_img_path = 'my_cat.jpg'
# sample_img_path = 'sample_dog.jpg'
# sample_img_path = 'sample_cat.jpg'
sample_img = load_img(sample_img_path, target_size=IMG_DIM)
sample_img_tensor = img_to_array(sample_img)
sample_img_tensor = np.expand_dims(sample_img_tensor, axis=0)
sample_img_tensor /= 255.

print(sample_img_tensor.shape)

In [ ]:
cnn_img_aug_prediction = num2class_label_transformer(img_aug_cnn.predict_classes(sample_img_tensor, verbose=0))
tlearn_cnn_finetune_img_aug_prediction = num2class_label_transformer(tl_img_aug_finetune_cnn.predict_classes(sample_img_tensor, verbose=0))
print('Predictions for our sample image:\n', 
      '\nCNN with Img Augmentation:', cnn_img_aug_prediction, 
      '\nPre-trained CNN with Fine-tuning & Img Augmentation (Transfer Learning):', tlearn_cnn_finetune_img_aug_prediction)

In [ ]:
img_aug_cnn.predict_proba(sample_img_tensor, verbose=0)

In [ ]:
tl_img_aug_finetune_cnn.predict_proba(sample_img_tensor, verbose=0)

## 2) CNN 모델이 인지한 것을 시각화하기

In [ ]:
tl_img_aug_finetune_cnn.layers[0].layers[1:9]

In [ ]:
from keras import models  

In [ ]:
# 최상위 8개 층에서 추출된 출력:
layer_outputs = [layer.output for layer in tl_img_aug_finetune_cnn.layers[0].layers[1:9]]

# 모델 입력에 따라 출력을 반환하는 모델을 생성:
activation_model = models.Model(inputs=tl_img_aug_finetune_cnn.layers[0].layers[1].input, outputs=layer_outputs)

# 층마다 배열이 1개이므로 numpy 배열 형태로 8개의 리스트를 반환
activations = activation_model.predict(sample_img_tensor)

In [ ]:
print ('Sample layer shape:', activations[0].shape)
print('Sample convolution (activation map) shape:', activations[0][0, :, :, 1].shape)

In [ ]:
## 샘플 특성 지도 추출
fig, ax = plt.subplots(1,5, figsize=(16, 6))
ax[0].imshow(activations[0][0, :, :, 10], cmap='bone')
ax[1].imshow(activations[0][0, :, :, 25], cmap='bone')
ax[2].imshow(activations[0][0, :, :, 40], cmap='bone')
ax[3].imshow(activations[0][0, :, :, 55], cmap='bone')
ax[4].imshow(activations[0][0, :, :, 63], cmap='bone')

In [ ]:
# These are the names of the layers, so can have them as part of our plot
layer_names = []
for layer in tl_img_aug_finetune_cnn.layers[0].layers[1:9]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, activations):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='bone')

- 최상위 층의 특성지도가 원래의 이미지를 더 많이 유지.
- 깊은 층 일수록 특성지도는 더 추상적이고 복잡해지며 해석이 어려워짐.

## 3) 테스트 데이터로 수행한 평가 모델 성능

In [ ]:
IMG_DIM = (150, 150)

test_files = glob.glob('test_data/*')
test_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in test_files]
test_imgs = np.array(test_imgs)
test_labels = [fn.split('/')[1].split('.')[0].strip() for fn in test_files]

test_labels_enc = class2num_label_transformer(test_labels)

print('Test dataset shape:', test_imgs.shape)

In [ ]:
## 데이터 노말라이즈(스케일링) 및 레이블 인코딩
test_imgs_scaled = test_imgs.astype('float32')
test_imgs_scaled /= 255

test_labels_enc = class2num_label_transformer(test_labels)

print(test_labels[0:5], test_labels_enc[0:5])

In [ ]:
## 학습 할때처럼 generator를 활용하여 예측하는 것도 가능.
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow(test_imgs, test_labels_enc, batch_size=batch_size, shuffle=False)
# predictions = img_aug_cnn.predict_generator(test_generator, verbose=0)

In [ ]:
## 주어진 데이터만으로 학습한 CNN 모델 (img_aug_cnn) 성능
predictions = img_aug_cnn.predict_classes(test_imgs_scaled, verbose=0)
predictions = num2class_label_transformer(predictions)

meu.display_model_performance_metrics(true_labels=test_labels, predicted_labels=predictions, classes=list(set(test_labels)))

In [ ]:
## VGG-16기반으로 전이학습한 CNN 모델 (tl_img_aug_finetune_cnn) 성능
predictions = tl_img_aug_finetune_cnn.predict_classes(test_imgs_scaled, verbose=0)
predictions = num2class_label_transformer(predictions)

meu.display_model_performance_metrics(true_labels=test_labels, predicted_labels=predictions, classes=list(set(test_labels)))

In [ ]:
## ROC 커브 비교
meu.plot_model_roc_curve(img_aug_cnn, test_imgs_scaled, 
                                  true_labels=test_labels_enc, class_names=[0, 1])

meu.plot_model_roc_curve(tl_img_aug_finetune_cnn, test_imgs_scaled, 
                                  true_labels=test_labels_enc, class_names=[0, 1])